## Understanding and Predicting Property Maintenance Fines

This project is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this project, the task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). 

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of     the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
    state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant

### Import the Packages

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score,accuracy_score

# Data Warngling

### Load the dataset

In [2]:
train_data = pd.read_csv("readonly/train.csv")
train_data.head()

C:\Users\Salah Mohamed\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (11,12,31) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
3,22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
4,22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN


In [3]:
train_data.nunique()

ticket_id                     250306
agency_name                        5
inspector_name                   173
violator_name                 119992
violation_street_number        19175
violation_street_name           1791
violation_zip_code                 0
mailing_address_str_number     15826
mailing_address_str_name       37896
city                            5184
state                             59
zip_code                        5642
non_us_str_code                    2
country                            5
ticket_issued_date             86979
hearing_date                    6222
violation_code                   235
violation_description            258
disposition                        9
fine_amount                       43
admin_fee                          2
state_fee                          2
late_fee                          37
discount_amount                   13
clean_up_cost                      1
judgment_amount                   57
payment_amount                   533
b

In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250306 entries, 0 to 250305
Data columns (total 34 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   ticket_id                   250306 non-null  int64  
 1   agency_name                 250306 non-null  object 
 2   inspector_name              250306 non-null  object 
 3   violator_name               250272 non-null  object 
 4   violation_street_number     250306 non-null  float64
 5   violation_street_name       250306 non-null  object 
 6   violation_zip_code          0 non-null       float64
 7   mailing_address_str_number  246704 non-null  float64
 8   mailing_address_str_name    250302 non-null  object 
 9   city                        250306 non-null  object 
 10  state                       250213 non-null  object 
 11  zip_code                    250305 non-null  object 
 12  non_us_str_code             3 non-null       object 
 13  country       

In [5]:
train_data['agency_name'].value_counts()

Buildings, Safety Engineering & Env Department    157784
Department of Public Works                         74717
Health Department                                   8903
Detroit Police Department                           8900
Neighborhood City Halls                                2
Name: agency_name, dtype: int64

### Drop two rows of 'Neighborhood City Halls' agency

In [6]:
data_clean = train_data[train_data['agency_name'] != 'Neighborhood City Halls']
data_clean['agency_name'].value_counts()

Buildings, Safety Engineering & Env Department    157784
Department of Public Works                         74717
Health Department                                   8903
Detroit Police Department                           8900
Name: agency_name, dtype: int64

### Drop null values of state column

In [7]:
data_clean.dropna(subset=['state'],inplace = True)
len(data_clean)

<ipython-input-7-21ca13fd2236>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean.dropna(subset=['state'],inplace = True)


250211

### Drop Not responsible rows from data set

In [8]:
data_clean.dropna(subset=['compliance'],inplace = True)
len(data_clean)

<ipython-input-8-effff2c99f49>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean.dropna(subset=['compliance'],inplace = True)


159795

### Choose the most suitable features to train the model

In [9]:
data_clean = data_clean.iloc[:,[1,10,14,15,18,19,20,21,22,23,24,25,26,27,33]]
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159795 entries, 0 to 250293
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   agency_name         159795 non-null  object 
 1   state               159795 non-null  object 
 2   ticket_issued_date  159795 non-null  object 
 3   hearing_date        159568 non-null  object 
 4   disposition         159795 non-null  object 
 5   fine_amount         159795 non-null  float64
 6   admin_fee           159795 non-null  float64
 7   state_fee           159795 non-null  float64
 8   late_fee            159795 non-null  float64
 9   discount_amount     159795 non-null  float64
 10  clean_up_cost       159795 non-null  float64
 11  judgment_amount     159795 non-null  float64
 12  payment_amount      159795 non-null  float64
 13  balance_due         159795 non-null  float64
 14  compliance          159795 non-null  float64
dtypes: float64(10), object(5)
memory u

### Edit Date columns to use as features

In [10]:
data_clean['hearing_date'].fillna(method = 'ffill',inplace=True)
data_clean['compliance'].fillna(2,inplace=True)
data_clean['fine_amount'].fillna(method = 'ffill',inplace=True)
data_clean.isna().sum()

agency_name           0
state                 0
ticket_issued_date    0
hearing_date          0
disposition           0
fine_amount           0
admin_fee             0
state_fee             0
late_fee              0
discount_amount       0
clean_up_cost         0
judgment_amount       0
payment_amount        0
balance_due           0
compliance            0
dtype: int64

In [11]:
data_clean['ticket_issued_date'] = pd.to_datetime(data_clean['ticket_issued_date'],format="%Y-%m-%d")
data_clean['hearing_date'] = pd.to_datetime(data_clean['hearing_date'],format="%Y-%m-%d")
data_clean[['ticket_issued_date','hearing_date']].head()

,ticket_issued_date,hearing_date
0,2004-03-16 11:40:00,2005-03-21 10:30:00
1,2004-04-23 12:30:00,2005-05-06 13:30:00
5,2004-05-01 11:50:00,2005-03-21 10:30:00
6,2004-06-14 14:15:00,2005-02-22 15:00:00
7,2004-06-16 12:30:00,2005-02-22 15:00:00


In [13]:
data_clean['ticket_issued_year'] = data_clean['ticket_issued_date'].dt.year
data_clean['ticket_issued_month'] = data_clean['ticket_issued_date'].dt.month
data_clean['ticket_issued_day'] = data_clean['ticket_issued_date'].dt.day

data_clean['hearing_date_year'] = data_clean['hearing_date'].dt.year
data_clean['hearing_date_month'] = data_clean['hearing_date'].dt.month
data_clean['hearing_date_day'] = data_clean['hearing_date'].dt.day
data_clean.dtypes

agency_name                    object
state                          object
ticket_issued_date     datetime64[ns]
hearing_date           datetime64[ns]
disposition                    object
fine_amount                   float64
admin_fee                     float64
state_fee                     float64
late_fee                      float64
discount_amount               float64
clean_up_cost                 float64
judgment_amount               float64
payment_amount                float64
balance_due                   float64
compliance                    float64
ticket_issued_year              int64
ticket_issued_month             int64
ticket_issued_day               int64
hearing_date_year               int64
hearing_date_month              int64
hearing_date_day                int64
dtype: object

### Downsampling the dataset

In [14]:
true_label = data_clean[data_clean['compliance'] == 1]
false_label = data_clean[data_clean['compliance'] == 0]

In [16]:
from sklearn.utils import resample
true_downsampled = resample(true_label,replace=False,n_samples=10000,random_state=42)
false_downsampled = resample(false_label,replace=False,n_samples=10000,random_state=42)

In [17]:
data_clean = pd.concat([true_downsampled,false_downsampled])
data_clean.shape

(20000, 21)

### Specifie target column and Features columns

In [18]:
y = data_clean['compliance'].astype('int64')
data_clean.drop(['ticket_issued_date','hearing_date','compliance'],axis = 1,inplace=True)
X = pd.get_dummies(data_clean,columns=['agency_name','state','disposition'])

### Build an intial random froest decision tree

In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

    
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=0)
    
RF = RandomForestClassifier(max_depth=8,n_estimators=800).fit(X_train,y_train)
    

y_pred = RF.predict(X_test)
train_score = RF.score(X_train,y_train)
test_score = RF.score(X_test,y_test)
score = roc_auc_score(y_test,y_pred)
print("the train score is :{:.2f} ".format(train_score))
print("the test score is :{:.2f}".format(test_score))
print("the area under the curve score is :{:.2f}".format(score))

the train score is :0.93 
the test score is :0.92
the area under the curve score is :0.92


### Use Grid scearch to find the optimel pramters

In [28]:
grad_pram = {'max_depth':[8,16,24,32],
            'n_estimators':[100,500,1000,2000]}
    
GS = GridSearchCV(RandomForestClassifier(),grad_pram,cv = 5,scoring='accuracy')
GS.fit(X_train,y_train)
best_pram = GS.best_params_
GR = RandomForestClassifier(max_depth=best_pram.get('max_depth'),\
                            n_estimators=best_pram.get('n_estimators')).fit(X_train,y_train)

y_predict = RF.predict(X_test)
grid_score = roc_auc_score(y_test,y_pred)
print("the best max depth is {} /n the best number of estimtors is {} \n"\
      .format(best_pram.get('max_depth'),best_pram.get('n_estimators')))
print("the area under the curve score is :{:.2f}".format(grid_score))

the best max depth is 24 /n the best number of estimtors is 500 

the area under the curve score is :0.92
